Based on [this notebook](https://www.kaggle.com/its7171/lgbm-with-loop-feature-engineering)

It seems that iterating over `.itertuples()` is slightly faster than over `.to_numpy()` and `.values`.

In [ ]:
import gc
from collections import defaultdict

import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from tqdm.notebook import tqdm

## setting
CV files are generated by [this notebook](https://www.kaggle.com/its7171/cv-strategy)

In [ ]:
train_pickle = '../input/riiid-cross-validation-files/cv1_train.pickle'
valid_pickle = '../input/riiid-cross-validation-files/cv1_valid.pickle'
question_file = '../input/riiid-test-answer-prediction/questions.csv'
debug = False
validaten_flg = False

## Benchmarks
Reference: https://github.com/chiphuyen/just-pandas-things/blob/master/just-pandas-things.ipynb

### Functions

In [ ]:
# funcs for user stats with loop
def add_user_feats_values(df, answered_correctly_sum_u_dict, count_u_dict):
    acsu = np.zeros(len(df), dtype=np.int32)
    cu = np.zeros(len(df), dtype=np.int32)
    for cnt,row in enumerate(tqdm(df[['user_id','answered_correctly']].values)):
        acsu[cnt] = answered_correctly_sum_u_dict[row[0]]
        cu[cnt] = count_u_dict[row[0]]
        answered_correctly_sum_u_dict[row[0]] += row[1]
        count_u_dict[row[0]] += 1
    user_feats_df = pd.DataFrame({'answered_correctly_sum_u':acsu, 'count_u':cu})
    user_feats_df['answered_correctly_avg_u'] = user_feats_df['answered_correctly_sum_u'] / user_feats_df['count_u']
    df = pd.concat([df, user_feats_df], axis=1)
    return df

In [ ]:
# funcs for user stats with loop
def add_user_feats_numpy(df, answered_correctly_sum_u_dict, count_u_dict):
    acsu = np.zeros(len(df), dtype=np.int32)
    cu = np.zeros(len(df), dtype=np.int32)
    for cnt,row in enumerate(tqdm(df[['user_id','answered_correctly']].to_numpy())):
        acsu[cnt] = answered_correctly_sum_u_dict[row[0]]
        cu[cnt] = count_u_dict[row[0]]
        answered_correctly_sum_u_dict[row[0]] += row[1]
        count_u_dict[row[0]] += 1
    user_feats_df = pd.DataFrame({'answered_correctly_sum_u':acsu, 'count_u':cu})
    user_feats_df['answered_correctly_avg_u'] = user_feats_df['answered_correctly_sum_u'] / user_feats_df['count_u']
    df = pd.concat([df, user_feats_df], axis=1)
    return df

In [ ]:

def add_user_feats_tuples(df, answered_correctly_sum_u_dict, count_u_dict):
    # the sum of questions answered correctly by the user (chronologically)
    acsu = np.zeros(len(df), dtype=np.int32)
    # the sum of questions answered by the user
    cu = np.zeros(len(df), dtype=np.int32)
    for cnt, row in enumerate(tqdm(df[['user_id','answered_correctly']].itertuples(), total=df.shape[0])):
        acsu[cnt] = answered_correctly_sum_u_dict[row.user_id]
        cu[cnt] = count_u_dict[row.user_id]
        answered_correctly_sum_u_dict[row.user_id] += row.answered_correctly
        count_u_dict[row.user_id] += 1
    user_feats_df = pd.DataFrame({'answered_correctly_sum_u':acsu, 'count_u':cu})
    user_feats_df['answered_correctly_avg_u'] = user_feats_df['answered_correctly_sum_u'] / user_feats_df['count_u']
    df = pd.concat([df, user_feats_df], axis=1)
    return df

In [ ]:

def add_user_feats_iterrows(df, answered_correctly_sum_u_dict, count_u_dict):
    # the sum of questions answered correctly by the user (chronologically)
    acsu = np.zeros(len(df), dtype=np.int32)
    # the sum of questions answered by the user
    cu = np.zeros(len(df), dtype=np.int32)
    for cnt, (_, row) in tqdm(enumerate(df[['user_id','answered_correctly']].iterrows()), total=df.shape[0]):
        acsu[cnt] = answered_correctly_sum_u_dict[row.user_id]
        cu[cnt] = count_u_dict[row.user_id]
        answered_correctly_sum_u_dict[row.user_id] += row.answered_correctly
        count_u_dict[row.user_id] += 1
    user_feats_df = pd.DataFrame({'answered_correctly_sum_u':acsu, 'count_u':cu})
    user_feats_df['answered_correctly_avg_u'] = user_feats_df['answered_correctly_sum_u'] / user_feats_df['count_u']
    df = pd.concat([df, user_feats_df], axis=1)
    return df

In [ ]:
# read data
def read_data():
    field_needed = ['row_id', 'user_id', 'content_id', 'content_type_id', 'answered_correctly', 'prior_question_elapsed_time', 'prior_question_had_explanation']
    train = pd.read_pickle(train_pickle)[field_needed]
    train = train.loc[train.content_type_id == False].reset_index(drop=True)
    return train

### .values

In [ ]:
%%time
train = read_data()

In [ ]:
%%time
answered_correctly_sum_u_dict = defaultdict(int)
count_u_dict = defaultdict(int)
train = add_user_feats_values(train, answered_correctly_sum_u_dict, count_u_dict)

In [ ]:
train.answered_correctly_sum_u.describe()

### .to_numpy()

In [ ]:
train = read_data()

In [ ]:
%%time
answered_correctly_sum_u_dict = defaultdict(int)
count_u_dict = defaultdict(int)
train = add_user_feats_numpy(train, answered_correctly_sum_u_dict, count_u_dict)

In [ ]:
train.answered_correctly_sum_u.describe()

### .itertuples()

In [ ]:
train = read_data()
gc.collect()

In [ ]:
%%time
answered_correctly_sum_u_dict = defaultdict(int)
count_u_dict = defaultdict(int)
train = add_user_feats_tuples(train, answered_correctly_sum_u_dict, count_u_dict)

In [ ]:
train.answered_correctly_sum_u.describe()

### .iterrows()

In [ ]:
train = read_data()
# Too slow so we have to take a sample (only 5%)
train = train.sample(frac=0.05)
gc.collect()

In [ ]:
%%time
answered_correctly_sum_u_dict = defaultdict(int)
count_u_dict = defaultdict(int)
train = add_user_feats_iterrows(train, answered_correctly_sum_u_dict, count_u_dict)

In [ ]:
train.answered_correctly_sum_u.describe()